In [400]:
from itertools import product
from random import randint, seed
import random
import numpy as np
from scipy import sparse
from functools import reduce
from random import random, choice, randint
from copy import copy
import math

In [401]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

Iterated Local Search

In [402]:
num_points = 5000
num_sets = num_points
density = .7
count_fitness = 0

mysets = make_set_covering_problem(num_points, num_sets, density).toarray()

def fitness(state):
    global count_fitness
    count_fitness = count_fitness + 1
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [mysets[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(num_points)]),
        )
    )
    return valid, -cost


In [403]:
def tweak(state):
    new_state = copy(state)
    index = randint(0, num_points - 1)
    new_state[index] = not new_state[index]
    return new_state

def perturb(state):
    num_changes = np.sum(state) * .25
    new_state = copy(state)

    for _ in range(int(num_changes)):
        index = randint(0, num_points - 1)
        new_state[index] = not state[index]

    return new_state


def new_home_state(current_home_base, current_state):
    if fitness(current_state) > fitness(current_home_base):
        return current_state
    else: return current_home_base
    

In [404]:
current_state = [choice([False, False, False, False, False, False]) for _ in range(num_sets)]

current_home_base = current_state
current_state_fitness = fitness(current_state)
timer = 0

while (fitness(current_home_base)[1] <=  fitness(current_state_fitness)[1]) and (timer <= 200):
    timer += 1
    for _ in range(100):
        new_state = tweak(current_state)
        if (fitness(new_state) > current_state_fitness):
            print(fitness(current_state))
            current_state = new_state 
            current_state_fitness = fitness(current_state) # in order to not compute it each time...

    current_home_base = new_home_state(current_home_base, current_state)
    current_state = perturb(current_home_base)

print("Fitness of the current_home_base")
print(fitness(current_home_base))
print(count_fitness)


(0, 0)
(3483, -1)
(4534, -2)
(4853, -3)
(4951, -4)
(4986, -5)
(4995, -6)
(4998, -7)
(4999, -8)
Fitness of the current_home_base
(5000, -9)
126
